In [16]:
# コメント
import random

# 親クラスの作成：プレイヤーとモンスターの共通部分を設定する
class Character:
    def __init__(self, name, HP, attack_power, MP, magic_power): # インスタンス化で使う属性の設定をする
        self.name   = name
        self.hp     = HP
        self.attack = attack_power
        self.mp = MP
        self.magic = magic_power
        self.max_hp = HP

    # モンスター・プレイヤー共通、物理攻撃3パターン  （プレイヤー・モンスターの共通の攻撃処理は親クラスに書く。誰を攻撃するかは引数で渡す）
    def common_attack(self, target):
        physical_attack_list =  [  1,   2,   3]  # 物理攻撃3パターン
        weights              =  [0.1, 0.2, 0.7]  # それぞれが出る確率
        physical_attack_type = random.choices(physical_attack_list, weights)[0] # ※重み（確率）を使いたい場合はchoices。返り値がリストなので [0] をつける

        if physical_attack_type == 1:    # 1なら攻撃力2倍
            damage = self.attack * 2
            damage_text = f"{self.name}の攻撃。会心の一撃！{target.name}に{damage}のダメージ。"

        elif physical_attack_type == 2:  # 2なら攻撃ミス
            damage = 0
            damage_text = f"{self.name}の攻撃。ミス！{target.name}は攻撃を受けない。"

        else:                            # 3は規定の攻撃をする
            damage = self.attack
            damage_text = f"{self.name}の攻撃。{target.name}に{damage}のダメージ。"

        target.hp -= damage # 残りのHPの計算
        return damage_text  # ダメージなどに関するテキストを返す
        # モンスター・プレイヤー共通、物理攻撃おわり


    # モンスター・プレイヤー共通、ホイミ
    def common_hoimi(self, heal_target): # △回復対象は各々で選ぶ
        if self.mp < 3:           # まずMP確認。足りなかったら、足りないことを表示して、何もせずに終わる
            return f"{self.name}はホイミを唱えた。MPが足りない！(現在MP:{self.mp})"
        else:
            self.mp -= 3                            # ホイミ：MP減らす
            heal = self.magic * 6                   # ホイミ：回復
            heal_target.hp += heal                  # 回復後のHPの計算
            if heal_target.hp > heal_target.max_hp: # 最大HPを超えないように調整
                heal_target.hp = heal_target.max_hp
            return f"{self.name}はホイミを唱えた。{heal_target.name}のHPは{heal_target.hp}に回復した。" # 回復のテキストを返す
            # モンスター・プレイヤー共通、ホイミ


# モンスタークラスの作成（Characterを継承しているので、Character のメソッドも使える）
class Monster(Character):
    def atack_type(self, players):   #　下記のループ処理で引数を統一させたいため作成
        pass  # メソッドは存在するけど、親クラスでは具体的な処理は書かないときに passを使う
              # ※引数playersがあるモンスターとないモンスターがいるが、ループする時は統一したいので、
              # ※atack_type(players)にしておく。atack_type(self, players = None)にすれば引数がなくても引数を使わない処理になる

    # モンスター共通の物理攻撃
    def do_attack(self):
        target = random.choice(players)   # ターゲットにするプレイヤーを決める
        common_attack_text = self.common_attack(target)

        if target.hp <= 0 :   # プレイヤーのHPが0より少なくなったら0に修正し、ダメージなどに関するテキストを返す
            target.hp = 0
            return f"{common_attack_text}{target.name}のHPは{target.hp}。{target.name}は気絶した＞＜。。" # △プレイヤーとの表現が違うのでここに書く
        else:
            return f"{common_attack_text}{target.name}のHPは{target.hp}。"
        # モンスター共通の物理攻撃物理攻撃おわり

    # モンスター共通のホイミ
    def hoimi(self, monsters):
        heal_target = random.choice(monsters)  # 回復するターゲットを決める
        common_heal_text = self.common_hoimi(heal_target)
        return common_heal_text                # 回復のテキストを返す
        # モンスター共通のホイミ


# 各モンスターの攻撃方法　3匹
# モンスター1　ドラキー
class Doraki(Monster):
    def atack_type(self, players = None):
        return self.do_attack()    # モンスター共通の物理攻撃のみ
   # ドラキーの攻撃処理おわり

# モンスター2　キメラ
class Kimera(Monster):
    def atack_type(self, players):
        # 物理攻撃か魔法（ギラ）か決める
        kimera_atack_type = random.randint(1,2)

        if kimera_atack_type == 1:     # 物理攻撃
            return self.do_attack()    # モンスター共通の物理攻撃

        elif kimera_atack_type == 2:   # 魔法（ギラ）なら
            if self.mp < 8:            # MP確認。足りなかったら、足りないことを表示して、攻撃せずに終わる
                return(f"{self.name}はギラを唱えた。MPが足りない！(現在MP: {self.mp})")
            else:
                self.mp -= 8             # ギラ：MP減らす
                damage = self.magic * 4
                gira_text = f"\n{self.name}はギラを唱えた。プレイヤー全員に{damage}のダメージ。\n"

                for player in players:
                    player.hp -= damage    # 各プレイヤーの残りのHPの計算

                    if player.hp <= 0:     # ターゲットのモンスターのHPが0より少なくなったら0に修正し、ダメージなどに関するテキストを返す
                        player.hp = 0
                        gira_text += f"{player.name}のHPは{player.hp}。{player.name}は気絶した＞＜。。\n"
                    else:
                        gira_text += f"{player.name}のHPは{player.hp}。\n"

                return gira_text
                # キメラの攻撃処理終わり

# モンスター3　ホイミスライム
class h_suraimu(Monster):
    def atack_type(self, players = None):
        # 物理攻撃か回復か決める
        suraimu_atack_type = random.randint(1,2)

        if suraimu_atack_type == 1:     # 物理攻撃
            return self.do_attack()     # モンスター共通の物理攻撃

        elif suraimu_atack_type == 2:   # ホイミなら、
            return self.hoimi(monsters) # モンスター共通のホイミ
  # ホイミスライムの攻撃処理終わり


# プレイヤークラスの作成
class Player(Character):
    def atack_type(self, monsters):  #　下記のループ処理で引数を統一させたいため作成
        pass

    # プレイヤー共通の物理攻撃
    def do_attack(self):
        target = random.choice(monsters)  # ターゲットにするモンスターを決める

        common_attack_text = self.common_attack(target)
        if target.hp <= 0 :               # ターゲットモンスターのHPが0より少なくなったら0に修正し、ダメージなどに関するテキストを返す
            target.hp = 0
            return f"{common_attack_text}{target.name}のHPは{target.hp}。{target.name}を倒した。" # △モンスターとの表現が違うのでここに書く
        else:
            return f"{common_attack_text}{target.name}のHPは{target.hp}。"
        # プレイヤー共通の物理攻撃おわり

    # プレイヤー共通のホイミ
    def hoimi(self, players):
        #heal_target_choice = int(input("1：騎士、2：攻撃魔法使い、3：回復魔法使い　選択→"))
        heal_target_choice = random.randint(1,3)    # 回復のターゲットを決める
        if heal_target_choice == 1:
            heal_target = knight
        elif heal_target_choice == 2:
            heal_target = wizardA
        elif heal_target_choice == 3:
            heal_target = wizardB

        common_heal_text = self.common_hoimi(heal_target)
        return common_heal_text   # 回復のテキストを返す
        # プレイヤー共通のホイミおわり

# 各プレイヤーの攻撃方法　3人
# 1.騎士
class Knight(Player):
    def atack_type(self, monsters=None):
        # 物理攻撃か回復か決める
        #knight_atack_types = int(input("1：攻撃、2：回復　選択→"))
        knight_atack_type = random.randint(1,2)

        if knight_atack_type == 1:     # 物理攻撃なら、
            return self.do_attack()    # プレイヤー共通の物理攻撃

        elif knight_atack_type == 2:   # 回復なら、
            return self.hoimi(players) # プレイヤー共通の回復の処理
        # 騎士の攻撃処理終わり

# 2.fire魔法使い（攻撃）
class WizardA(Player):
    # 物理攻撃か魔法攻撃か決める
    def atack_type(self, monsters):
        #wizardA_atack_types = int(input("1：攻撃、2：魔法攻撃　選択→"))
        wizarA_atack_type = random.randint(1,2) # 攻撃方法を決める

        # 物理攻撃
        if wizarA_atack_type == 1:   # 物理攻撃なら、
            return self.do_attack()  # プレイヤー共通の物理攻撃
        # 物理攻撃おわり


        # 魔法攻撃(メラとイオ)か決める
        elif wizarA_atack_type == 2:
            #magic_types = int(input("1：メラ、2：イオ　選択→"))
            magic_types = random.randint(1,2)

            # メラ
            if magic_types == 1:
                if self.mp < 2:             # MP確認。足りなかったら、足りないことを表示して、攻撃せずに終わる
                    return(f"{self.name}はメラを唱えた。MPが足りない！(現在MP: {self.mp})")
                else:
                    target = random.choice(monsters)  # ターゲットにするモンスターを決める
                    self.mp -= 2                      # メラ：MP減らす
                    damage = self.magic * 2
                    magic_damage_text = f"{self.name}はメラを唱えた。{target.name}に{damage}のダメージ"
                    target.hp -= damage # モンスターの残りのHPの計算

                    if target.hp <= 0:  # ターゲットのモンスターのHPが0より少なくなったら0に修正し、ダメージなどに関するテキストを返す
                        target.hp = 0
                        return f"{magic_damage_text}。{target.name}のHPは{target.hp}。{target.name}を倒した"
                    else:
                        return f"{magic_damage_text}。{target.name}のHPは{target.hp}。"
                    # メラおわり

            # イオ
            elif magic_types == 2:
                if self.mp < 10:         # MP確認。足りなかったら、足りないことを表示して、攻撃せずに終わる
                    return(f"{self.name}はイオを唱えた。MPが足りない！(現在MP: {self.mp})")
                else:
                    self.mp -= 10            # イオ：MP減らす
                    damage = self.magic * 4
                    io_text = f"{self.name}はイオを唱えた。モンスター全員に{damage}のダメージ。\n"

                    for monster in monsters:
                        monster.hp -= damage    # 各モンスターの残りのHPの計算

                        if monster.hp <= 0:     # ターゲットのモンスターのHPが0より少なくなったら0に修正し、ダメージなどに関するテキストを返す
                            monster.hp = 0
                            io_text += f"{monster.name}のHPは{monster.hp}。{monster.name}を倒した。\n"
                        else:
                            io_text += f"{monster.name}のHPは{monster.hp}。\n"

                    return io_text
                    # イオ終わり

# 3.魔法使い（回復）
class WizardB(Player):
    # 物理攻撃か回復か決める
    def atack_type(self, monsters=None):
        #wizardB_atack_types = int(input("1：攻撃、2：回復　選択→"))
        wizarB_atack_type = random.randint(1,2)

        # 物理攻撃
        if wizarB_atack_type == 1:
            return self.do_attack()    # プレイヤー共通の物理攻撃
        # 物理攻撃おわり


        # 回復魔法(ホイミかベホマラー)を決める
        elif wizarB_atack_type == 2:
            #magic_types = int(input("1：ホイミ、2：ベホマラー　選択→"))
            magic_types = random.randint(1,2)

            # ホイミ
            if magic_types == 1:
                return self.hoimi(players) # プレイヤー共通のホイミ
            # ホイミ終わり


            #ベホマラー
            elif magic_types == 2:
                if self.mp < 10:          # MP確認。足りなかったら、足りないことを表示して、回復せずに終わる
                    return(f"{self.name}はベホマラーを唱えた。MPが足りない！(現在MP: {self.mp})")
                else:
                    self.mp -= 10          # ベホマラー：MP減らす
                    heal = self.magic * 6
                    behoma_text = f"{self.name}はベホマラーを唱えた。\n"

                    for player in players:
                        player.hp += heal             # 各プレイヤーの残りのHPの計算

                        if player.hp > player.max_hp: # 最大HPを超えないようにする
                            player.hp = player.max_hp

                        behoma_text += f"{player.name}のHPは{player.hp}に回復した。\n" # 回復のテキストを返す
                    return behoma_text
                    # ベホマラーおわり


# 生存判定
def alive_judge(players, monsters):
    if not monsters:  # モンスターリストに誰もいない時
        print("\n！！敵をすべて倒した！！")
        return True
    elif not players: # プレイヤーに誰もいない時
        print("\n～ゲームオーバー～")
        return True
    else:
        return False

# モンスターの情報（名前、HP、攻撃力、MP）
kimeraA    = Kimera   ("キメラA",        50, 22, 10, 3)
kimeraB    = Kimera   ("キメラB",        50, 22, 10, 3)
h_suraimuA = h_suraimu("ホイミスライムA",30, 18, 30, 5)
h_suraimuB = h_suraimu("ホイミスライムB",30, 18, 30, 5)
dorakiA    = Doraki   ("ドラキー強A",    25, 16,  0, 0)
dorakiB    = Doraki   ("ドラキー強B",    25, 16,  0, 0)

# プレイヤーの情報
knight   = Knight ("騎士",        180, 15,  6, 5)
wizardA  = WizardA("fire魔法使い",160,  8, 20, 5)
wizardB  = WizardB("holy魔法使い",140,  6, 20, 5)

# ループ処理のためにモンスターとプレイヤーのリスト
monsters = [kimeraA, kimeraB, h_suraimuA, h_suraimuB, dorakiA, dorakiB]
players  = [knight, wizardA, wizardB]

# ---------------------------------------------------------

# モンスターの出現情報
for monster in monsters:
     print(f"{monster.name}が現れた。")

turn = 0 # ターン数表示のため
while players and monsters:   # リストにプレイヤーもモンスターもいる時は繰り返す（monstersと書くだけで誰かがいることを指す）
    turn += 1
    print(f"\n●{turn}ターン目●")

    # プレイヤーたちの攻撃
    random.shuffle(players)                 # プレイヤーが攻撃する順番をランダムにする
    for player in players:
        print(f"\n☆{player.name}の番☆")
        print(player.atack_type(monsters))  # プレイヤーの攻撃を実行

        # 倒したモンスターを表示させないように、攻撃するたびにリストを更新
        monsters = [monster for monster in monsters if monster.hp > 0]
        # △攻撃前だけに置くと、倒したモンスターのリスト更新ができていないので倒したのに攻撃してくる

        # モンスターを全員倒したかどうか確認
        if alive_judge(players, monsters): # △Trueなら抜ける
            break

    # モンスター達の攻撃
    #print(f"\n!!モンスターの攻撃の番!!")
    random.shuffle(monsters)                 # モンスターが攻撃する順番をランダムにする

    for monster in monsters:
        # players = [player for player in players if player.hp > 0] # プレイヤーリスト更新
        # if not players:  # プレイヤーがいなければ終了（これがないとターゲットを選べず止まる）
        #   break
        print(f"\n!!{monster.name}の番!!")
        print(monster.atack_type(players))  # モンスターの攻撃を実行。

        # 倒したプレイヤーを表示させないように、攻撃するたびにリストを更新
        players = [player for player in players if player.hp > 0] # プレイヤーリスト更新

        # プレイヤーが全員倒されたかどうか確認
        if alive_judge(players, monsters):
            break


キメラAが現れた。
キメラBが現れた。
ホイミスライムAが現れた。
ホイミスライムBが現れた。
ドラキー強Aが現れた。
ドラキー強Bが現れた。

●1ターン目●

☆fire魔法使いの番☆
fire魔法使いの攻撃。ホイミスライムBに8のダメージ。ホイミスライムBのHPは22。

☆holy魔法使いの番☆
holy魔法使いはベホマラーを唱えた。
fire魔法使いのHPは160に回復した。
holy魔法使いのHPは140に回復した。
騎士のHPは180に回復した。


☆騎士の番☆
騎士はホイミを唱えた。fire魔法使いのHPは160に回復した。

!!ホイミスライムAの番!!
ホイミスライムAの攻撃。騎士に18のダメージ。騎士のHPは162。

!!ドラキー強Bの番!!
ドラキー強Bの攻撃。fire魔法使いに16のダメージ。fire魔法使いのHPは144。

!!キメラBの番!!
キメラBの攻撃。holy魔法使いに22のダメージ。holy魔法使いのHPは118。

!!キメラAの番!!

キメラAはギラを唱えた。プレイヤー全員に12のダメージ。
fire魔法使いのHPは132。
holy魔法使いのHPは106。
騎士のHPは150。


!!ホイミスライムBの番!!
ホイミスライムBはホイミを唱えた。キメラBのHPは50に回復した。

!!ドラキー強Aの番!!
ドラキー強Aの攻撃。holy魔法使いに16のダメージ。holy魔法使いのHPは90。

●2ターン目●

☆騎士の番☆
騎士の攻撃。ミス！ホイミスライムBは攻撃を受けない。ホイミスライムBのHPは22。

☆holy魔法使いの番☆
holy魔法使いはベホマラーを唱えた。
騎士のHPは180に回復した。
holy魔法使いのHPは120に回復した。
fire魔法使いのHPは160に回復した。


☆fire魔法使いの番☆
fire魔法使いはイオを唱えた。モンスター全員に20のダメージ。
ホイミスライムAのHPは10。
ドラキー強BのHPは5。
キメラBのHPは30。
キメラAのHPは30。
ホイミスライムBのHPは2。
ドラキー強AのHPは5。


!!ホイミスライムAの番!!
ホイミスライムAの攻撃。騎士に18のダメージ。騎士のHPは162。

!!ドラキー強Bの番!!
ドラキー強Bの攻撃。会心の一撃！fire魔

In [ ]:
# Character（親クラス：共通機能）
# ├─ Player（プレイヤー共通）
# │   ├─ Knight（騎士：物理攻撃のみ）
# │   ├─ FireWizard（火の魔法専門）
# │   └─ HolyWizard（回復専門）※任意で細分化可能
# │
# └─Monster（モンスター共通）
#     ├── kimera（キメラ系）
#     │     └─ キメラA/B
#     ├── h_suraimu（ホイミスライム系）
#     │     └─ ホイミスライムA/B
#     └── dorak（ドラキー強系）
#     　　    └─ ドラキー強A/B